In [2]:
from huggingface_hub import login
login()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install Cython packaging
!pip install 'git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[asr]'


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [357 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,383 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,783 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,767 kB]
Get:13 https://ppa.launchpadcontent.net/deadsnakes/p

In [14]:
predicted_probs

[tensor([[[8.5176e-02, 1.5857e-04, 3.4543e-07, 1.9704e-08],
          [6.2547e-02, 7.5510e-05, 2.9786e-07, 1.9055e-08],
          [1.1647e-01, 9.1661e-05, 3.0476e-07, 1.9508e-08],
          ...,
          [6.3567e-01, 2.8266e-03, 1.6394e-07, 8.6371e-09],
          [9.2230e-01, 6.8071e-03, 6.4720e-08, 1.5036e-09],
          [9.5116e-01, 2.9255e-02, 1.0243e-07, 2.1588e-09]]])]

In [4]:
!pip install pyannote.audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 893.7/893.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.6/853.6 kB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.9 MB/s eta 0:00:00
   ━

In [7]:
import os
import sys
import torch
from pyannote.core import Annotation, Segment
from pyannote.metrics.diarization import DiarizationErrorRate

# --- LE MONKEY PATCH POUR NE MO (Obligatoire pour Sortformer sur r1.23.0) ---
from nemo.collections.asr.models import EncDecDiarLabelModel
import nemo.collections.asr.models.label_models as label_models
sys.modules['nemo.collections.asr.models.sortformer_diar_models'] = label_models
label_models.SortformerEncLabelModel = EncDecDiarLabelModel

# ==========================================
# 1. FONCTIONS DE PARSING
# ==========================================

def load_ground_truth_rttm(rttm_path):
    annotation = Annotation()
    if not os.path.exists(rttm_path):
        return annotation

    with open(rttm_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 8 and parts[0] == 'SPEAKER':
                start = float(parts[3])
                duration = float(parts[4])
                speaker = parts[7]
                annotation[Segment(start, start + duration)] = speaker
    return annotation

def parse_nemo_predictions(predicted_segments):
    annotation = Annotation()
    if isinstance(predicted_segments, list) and len(predicted_segments) > 0:
        segments_list = predicted_segments[0]
        for seg_str in segments_list:
            parts = seg_str.split()
            if len(parts) >= 3:
                start = float(parts[0])
                end = float(parts[1])
                speaker = parts[2]
                annotation[Segment(start, end)] = speaker
    return annotation

# ==========================================
# 2. CHARGEMENT DU MODÈLE
# ==========================================

print("⏳ Chargement du modèle Sortformer...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

diar_model = EncDecDiarLabelModel.from_pretrained("nvidia/diar_streaming_sortformer_4spk-v2")
diar_model.to(device)
diar_model.eval()
print(f"✅ Modèle chargé sur {device} !")

# ==========================================
# 3. CONFIGURATION DES DOSSIERS
# ==========================================

AUDIO_DIR = "/content/drive/MyDrive/GroundTruth_overlap_noise/GroundTruth_overlap_noise/audioMono/"
RTTM_DIR = "/content/drive/MyDrive/GroundTruth_overlap/rttm/"

# Configuration de la métrique Pyannote
der_metric = DiarizationErrorRate(collar=0.25, skip_overlap=False)

# ==========================================
# 4. BOUCLE SUR TOUS LES FICHIERS
# ==========================================

# On récupère tous les fichiers .wav et on les trie
audio_files = sorted([f for f in os.listdir(AUDIO_DIR) if f.endswith('.wav')])

if not audio_files:
    print(f"❌ Aucun fichier .wav trouvé dans {AUDIO_DIR}")
    exit()

print("\n🚀 Lancement de l'évaluation par lots...")
print("-" * 80)
print(f"{'FICHIER':<30} | {'DER (%)':<10} | {'VRAI':<5} | {'PRED':<5} | {'DIFF':<5}")
print("-" * 80)

# Variables pour compter les sur/sous estimations
stats_speakers = {"correct": 0, "sur_est": 0, "sous_est": 0}

for filename in audio_files:
    # Déduire les chemins complets
    file_base = os.path.splitext(filename)[0]
    audio_path = os.path.join(AUDIO_DIR, filename)
    rttm_path = os.path.join(RTTM_DIR, file_base + ".rttm")

    # Ignorer si le RTTM n'existe pas
    if not os.path.exists(rttm_path):
        print(f"{filename:<30} | ⚠️ IGNORÉ (RTTM introuvable)")
        continue

    # 1. Inférence NeMo
    try:
        predicted_segments = diar_model.diarize(audio=audio_path, batch_size=1)
    except Exception as e:
        print(f"{filename:<30} | ❌ ERREUR INFÉRENCE : {e}")
        continue

    # 2. Parsing et Métriques
    reference = load_ground_truth_rttm(rttm_path)
    hypothesis = parse_nemo_predictions(predicted_segments)

    if len(reference) == 0:
        continue

    # Calcul du DER pour CE fichier (et accumulation dans der_metric)
    der_val = der_metric(reference, hypothesis) * 100

    # Stats de locuteurs
    t_spk = len(reference.labels())
    p_spk = len(hypothesis.labels())
    diff = p_spk - t_spk

    if diff == 0: stats_speakers["correct"] += 1
    elif diff > 0: stats_speakers["sur_est"] += 1
    else: stats_speakers["sous_est"] += 1

    # Affichage ligne par ligne
    print(f"{filename:<30} | {der_val:.2f}%      | {t_spk:<5} | {p_spk:<5} | {diff:+d}")

# ==========================================
# 5. RÉSULTATS GLOBAUX
# ==========================================

# Le DER global se calcule via abs() sur la métrique accumulée
global_der = abs(der_metric) * 100

print("\n" + "="*50)
print(" 📊 BILAN GLOBAL (SORTFORMER)")
print("="*50)
print(f"DER Global (Pondéré) : {global_der:.2f}%")
print("-" * 50)
print("Estimation du nombre de locuteurs :")
print(f" ✅ Exacts       : {stats_speakers['correct']}")
print(f" 📈 Sur-estimés  : {stats_speakers['sur_est']}")
print(f" 📉 Sous-estimés : {stats_speakers['sous_est']}")
print("="*50)

⏳ Chargement du modèle Sortformer...


[NeMo W 2026-02-25 13:43:49 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    num_spks: 4
    session_len_sec: 90
    soft_label_thres: 0.5
    soft_targets: false
    labels: null
    batch_size: 4
    shuffle: true
    num_workers: 18
    validation_mode: false
    use_lhotse: false
    use_bucketing: false
    pin_memory: true
    window_stride: 0.01
    subsampling_factor: 8
    
[NeMo W 2026-02-25 13:43:49 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    is_tarred: false
    tarred_audio_filepaths: null
    sample_rate: 16000
    num_spks: 4
    sess

[NeMo I 2026-02-25 13:43:50 save_restore_connector:286] Model SortformerEncLabelModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--diar_streaming_sortformer_4spk-v2/snapshots/6dbf0d69730bfee097056692b86525a0a23b32f9/diar_streaming_sortformer_4spk-v2.nemo.
✅ Modèle chargé sur cuda !

🚀 Lancement de l'évaluation par lots...
--------------------------------------------------------------------------------
FICHIER                        | DER (%)    | VRAI  | PRED  | DIFF 
--------------------------------------------------------------------------------
[NeMo I 2026-02-25 13:43:50 vad_utils:81] No postprocessing YAML file has been provided. Default postprocessing configurations will be applied.


[NeMo W 2026-02-25 13:43:50 dataloader:826] The following configuration keys are ignored by Lhotse dataloader: num_spks,soft_label_thres,session_len_sec
Diarizing: 0it [00:00, ?it/s]
Streaming Steps: 100%|██████████| 2/2 [00:00<00:00, 16.43it/s]
Diarizing: 1it [00:01,  1.23s/it]


hard_overlap_000.wav           | 61.84%      | 2     | 1     | -1
[NeMo I 2026-02-25 13:43:52 vad_utils:81] No postprocessing YAML file has been provided. Default postprocessing configurations will be applied.


...

hard_overlap_027.wav           | 13.52%      | 2     | 2     | +0
[NeMo I 2026-02-25 13:44:38 vad_utils:81] No postprocessing YAML file has been provided. Default postprocessing configurations will be applied.


[NeMo W 2026-02-25 13:44:38 dataloader:826] The following configuration keys are ignored by Lhotse dataloader: num_spks,soft_label_thres,session_len_sec
Diarizing: 0it [00:00, ?it/s]
Streaming Steps: 100%|██████████| 2/2 [00:00<00:00, 20.21it/s]
Diarizing: 1it [00:01,  1.25s/it]


hard_overlap_028.wav           | 19.85%      | 2     | 2     | +0
[NeMo I 2026-02-25 13:44:39 vad_utils:81] No postprocessing YAML file has been provided. Default postprocessing configurations will be applied.


[NeMo W 2026-02-25 13:44:39 dataloader:826] The following configuration keys are ignored by Lhotse dataloader: num_spks,soft_label_thres,session_len_sec
Diarizing: 0it [00:00, ?it/s]
Streaming Steps: 100%|██████████| 2/2 [00:00<00:00, 16.57it/s]
Diarizing: 1it [00:01,  1.08s/it]


hard_overlap_029.wav           | 10.24%      | 2     | 2     | +0

 📊 BILAN GLOBAL (SORTFORMER)
DER Global (Pondéré) : 25.20%
--------------------------------------------------
Estimation du nombre de locuteurs :
 ✅ Exacts       : 27
 📈 Sur-estimés  : 0
 📉 Sous-estimés : 3
